In [ ]:
import os
import zipfile
from google.colab import drive

drive.mount('/content/drive')

DRIVE_PROJECT_PATH = '/content/drive/MyDrive/skin_lession_classification'
ZIP_PATH = os.path.join(DRIVE_PROJECT_PATH, 'dataset.zip')

if not os.path.exists('/content/dataset'):
    print("Veri seti açılıyor")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall('/content/dataset')
    print("BAŞARILI! Veri seti '/content/dataset' klasörüne açıldı.")
else:
    print("Veri seti zaten hazır.")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Eğitim şu cihazda yapılacak: {device}")

DATA_DIR = '/content/dataset'
CLASSES = sorted(os.listdir(os.path.join(DATA_DIR, 'train')))
print(f"Sınıflar: {CLASSES}")

In [ ]:
print("--- 1. Deep Learning baseline (ResNet50) Başlıyor ---")

# Veri Dönüşümleri (Augmentation sadece Train'de var!)
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Veri Yükleyiciler (DataLoader)
image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

# Model Kurulumu (resnet50_weights.default en guncel agirlik dosyalarini bulacaktir.)
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(CLASSES))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

rn50_train_losses, rn50_val_losses = [], []
num_epochs = 10

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs} başlıyor...')

    for phase in ['train', 'val']:
        if phase == 'train': model.train()
        else: model.eval()

        running_loss = 0.0
        running_corrects = 0

        pbar = tqdm(dataloaders[phase], desc=f'{phase} Phase', leave=True)

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            current_loss = running_loss / ((pbar.n + 1) * inputs.size(0)) if pbar.n > 0 else loss.item()
            pbar.set_postfix({'loss': f'{current_loss:.4f}'})

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase.upper()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'train': rn50_train_losses.append(epoch_loss)
        else: rn50_val_losses.append(epoch_loss)

save_path = os.path.join(DRIVE_PROJECT_PATH, 'resnet50_d2_model.pth')
torch.save(model.state_dict(), save_path)
print(f"\nModel başarıyla kaydedildi: {save_path}")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(rn50_train_losses, label='Training Loss')
plt.plot(rn50_val_losses, label='Validation Loss')
plt.title('ResNet50 Eğitim ve Doğrulama Kaybı (Loss Curves)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

![alt text](../output/figures/resnet50_train_val_loss.png "resnet-50-train-val-loss-image")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

print("--- ResNet50 Test Seti Performans Analizi ---")

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'test'), test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
y_true = []
y_pred = []

print("Test seti üzerinde tahmin yapılıyor...")
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\n>>> ResNet50 Detaylı Sınıflandırma Raporu:")
print(classification_report(y_true, y_pred, target_names=CLASSES))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
plt.title('Confusion Matrix (Hata Matrisi)')
plt.ylabel('Gerçek Sınıf')
plt.xlabel('Tahmin Edilen Sınıf')
plt.show()


>>> ResNet50 Detaylı Sınıflandırma Raporu:
              precision    recall  f1-score   support

       akiec       0.80      0.50      0.62        32
         bcc       0.85      0.65      0.74        52
         bkl       0.78      0.73      0.75       110
          df       0.67      0.55      0.60        11
         mel       0.81      0.49      0.61       112
          nv       0.88      0.99      0.93       671
        vasc       1.00      0.86      0.92        14

    accuracy                           0.86      1002
   macro avg       0.83      0.68      0.74      1002
weighted avg       0.86      0.86      0.85      1002

![alt text](../output/figures/resnet50_conf_matrix.png "resnet-50-conf-matrix-image")

In [ ]:
print("--- 2. DL Model: Vision Transformer (ViT-B/16) Başlıyor ---")

model_vit = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)

num_ftrs = model_vit.heads.head.in_features
model_vit.heads.head = nn.Linear(num_ftrs, len(CLASSES))

model_vit = model_vit.to(device)

#ViT için Learning Rate biraz daha düşük olabilir
criterion = nn.CrossEntropyLoss()
optimizer_vit = optim.SGD(model_vit.parameters(), lr=0.001, momentum=0.9)

vit_train_losses, vit_val_losses = [], []
num_epochs = 10

for epoch in range(num_epochs):
    print(f'ViT Epoch {epoch+1}/{num_epochs} başlıyor...')

    for phase in ['train', 'val']:
        if phase == 'train': model_vit.train()
        else: model_vit.eval()

        running_loss = 0.0
        running_corrects = 0

        pbar = tqdm(dataloaders[phase], desc=f'{phase} Phase', leave=True)

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer_vit.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model_vit(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer_vit.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            current_loss = running_loss / ((pbar.n + 1) * inputs.size(0)) if pbar.n > 0 else loss.item()
            pbar.set_postfix({'loss': f'{current_loss:.4f}'})

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'   {phase.upper()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'train': vit_train_losses.append(epoch_loss)
        else: vit_val_losses.append(epoch_loss)

#save
torch.save(model_vit.state_dict(), os.path.join(DRIVE_PROJECT_PATH, 'vit_b16_d2_model.pth'))
print("ViT Modeli Kaydedildi!")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(vit_train_losses, label='Training Loss')
plt.plot(vit_val_losses, label='Validation Loss')
plt.title('Vit16 Eğitim ve Doğrulama Kaybı (Loss Curves)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

![alt text](../output/figures/vit16_train_val_loss.png "vit-16-train-val-loss-image")

In [ ]:
print("--- Vision Transformer (ViT) Test Seti Performans Analizi ---")

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'test'), test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# 2. ViT Modelini Değerlendirme Moduna Al
model_vit.eval()
y_true_vit = []
y_pred_vit = []

# 3. Tahminleri Topla
print("ViT ile Test seti üzerinde tahmin yapılıyor...")
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # DİKKAT: Burada model_vit kullanıyoruz
        outputs = model_vit(inputs)
        _, preds = torch.max(outputs, 1)

        y_true_vit.extend(labels.cpu().numpy())
        y_pred_vit.extend(preds.cpu().numpy())

# 4. Raporu Yazdır
print("\n>>> ViT (Vision Transformer) Detaylı Sınıflandırma Raporu:")
print(classification_report(y_true_vit, y_pred_vit, target_names=CLASSES))

# 5. Confusion Matrix Çiz
cm_vit = confusion_matrix(y_true_vit, y_pred_vit)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_vit, annot=True, fmt='d', cmap='Reds', xticklabels=CLASSES, yticklabels=CLASSES) # Rengi karışmasın diye Kırmızı yaptım
plt.title('ViT Confusion Matrix (Hata Matrisi)')
plt.ylabel('Gerçek Sınıf')
plt.xlabel('Tahmin Edilen Sınıf')
plt.show()

--- Vision Transformer (ViT) Test Seti Performans Analizi ---
ViT ile Test seti üzerinde tahmin yapılıyor...

>>> ViT (Vision Transformer) Detaylı Sınıflandırma Raporu:
              precision    recall  f1-score   support

       akiec       0.73      0.34      0.47        32
         bcc       0.82      0.63      0.72        52
         bkl       0.84      0.55      0.67       110
          df       0.75      0.55      0.63        11
         mel       0.64      0.80      0.71       112
          nv       0.90      0.96      0.93       671
        vasc       1.00      0.93      0.96        14

    accuracy                           0.86      1002
   macro avg       0.81      0.68      0.73      1002
weighted avg       0.86      0.86      0.85      1002

![alt text](../output/figures/vit16_conf_matrix.png "vit-16-conf-matrix-image")
